In [1]:
import pandas as pd
import numpy as np
import matplotlib
import os
import glob
import math
import shapefile as shp
import re
import copy
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from snapshot_selenium import snapshot as driver

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.render import make_snapshot
# driver = webdriver.Chrome(executable_path='C:/Python38/Scripts/chromedriver.exe')

In [2]:
users = pd.read_csv('users.csv')
users = users[['id', 'gender', 'location', 'fans', 'certified']]
print(len(users))
users = users.drop_duplicates()
print(len(users))

8260
6401


In [3]:
# we say if follower has passed 5000 then the account has some influence
users_fans = users
users['fans'] = users_fans['fans'].apply(lambda x: 0 if x <=5000 else 1)
# add new column
users['device'] = np.nan
users['ban'] = np.nan
users['ban_binary'] = np.nan
# print(users)

In [4]:
def findfiles(path):
    return glob.glob(path)

In [5]:
working_directory = os.getcwd() + '\\weibo_1\\'
weibo_list = findfiles(working_directory + '*')


In [6]:
# below is for grouping the device
for i in weibo_list:
    weibo_ = findfiles(i + '\\*.csv')
    id = os.listdir(i)
    id = os.path.splitext(os.path.basename(id[0]))[0]
    user_weibo = pd.read_csv(weibo_[0])
    device = str(user_weibo['工具'].mode()[0] if not user_weibo['工具'].mode().empty else 'unknown')
    users.loc[users['id'] == int(id), ['device']] = device


In [7]:
# below we process the banning
string1 = '抱歉，'
string2 = '该微博因被多人投诉'
string3 = '该账号因被投诉违反'

for i in weibo_list:
    weibo_ = findfiles(i + '\\*.csv')
    id =os.path.splitext(os.path.basename(os.listdir(i)[0]))[0]
    user_weibo = pd.read_csv(weibo_[0])
    count = 0
    for i in user_weibo['源微博正文']:
        if string1 in i or string2 in i or string3 in i:
            count += 1
    users.loc[users['id'] == int(id), ['ban']] = count

In [8]:
# add the binary classification of ban
users.loc[users['ban']>=1, ['ban_binary']] = True
users.loc[users['ban']<1, ['ban_binary']] = False

In [9]:
# below is the map processing
# we now only do class_one
location_ = [i for i in users['location']]
locations = [item[:2] if str(item)!= 'nan' else item == '123' for item in location_]

locations_ = []
for i in locations:
    if i != '黑龙':
        locations_.append(i)
    else:
        locations_.append('黑龙江')

In [10]:
users['location'] = locations_

In [11]:
# below we see how many unique values are there for location
users.loc[(users['location']) == False, ['location']] = 'unknown'

In [12]:
#map the data(preprocess)
users_map = users['location'].unique()
index = np.argwhere(users_map=='海外' )
index1 = np.argwhere(users_map == 'unknown')
index = np.append(index, index1)
users_map = np.delete(users_map, index)
count_location = users['location'].value_counts()
count_location = count_location.drop(labels=['海外', 'unknown', '其他'])
count_location = count_location.reset_index().values.tolist()
count_location = [tuple(i) for i in count_location]

In [13]:
# group by total number of bans based on location
count_location_ban = users.groupby('location')['ban'].count()
count_location_ban = count_location_ban.drop(labels = ['unknown', '其他','海外'])
count_location_ban = count_location_ban.reset_index().values.tolist()

count_location_ban = [tuple(i) for i in count_location_ban]

In [16]:
# mapping of locations based on number of bans respectively
# heat map
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
geo = (Geo().
        add_schema(maptype="china")
        .add("test",count_location_ban,type_=ChartType.HEATMAP)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(),
            legend_opts=opts.LegendOpts(is_show=False),
            title_opts=opts.TitleOpts(title="Heat map of ban users in absolute count"))
      )
geo.render('heat_map_ban.html')

'C:\\Users\\pasaa\\Desktop\\440_data_processing\\heat_map_ban.html'

In [15]:
# mapping of raw users' location
# heat map
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType
import random
geo = (Geo().
        add_schema(maptype="china")
        .add("test",count_location,type_=ChartType.HEATMAP)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(),
            legend_opts=opts.LegendOpts(is_show=False),
            title_opts=opts.TitleOpts(title="Heat map of raw users"))
      )
geo.render('heat_map_raw.html')

'C:\\Users\\pasaa\\Desktop\\440_data_processing\\heat_map_raw.html'

In [17]:
users = users.reset_index()
device = copy.copy(users['device'])

In [18]:
# grouping of devices
for i in range(len(device)):
    if any(x in str(device[i]) for x in ['iPhone','Mac']):
        device[i] = 'Apple'
    elif any(x in str(device[i]) for x in ['浏览器', '微博']):
        device[i] = 'web'
    elif any(x in str(device[i]) for x in ['小米', 'Redmi']):
        device[i] = 'Xiaomi'
    elif any(x in str(device[i]) for x in ['Huawei','nova','荣耀','华为','HUAWEI']):
        device[i] = 'Huawei'
    elif 'OPPO' in str(device[i]):
        device[i] = 'Oppo'
    elif 'Samsung' in str(device[i]):
        device[i] = 'Samsung'
    elif 'Nokia' in str(device[i]):
        device[i] = 'Nokia'
    elif 'vivo' in str(device[i]):
        device[i] = 'Vivo'
    else:
        device[i] = 'else'
users['device'] = device

In [19]:
# plot of raw_device
raw_device_count = users['device'].value_counts().drop(labels='else')
raw_device_count= raw_device_count.reset_index().values.tolist()
raw_device_count = [tuple(i) for i in raw_device_count]

In [24]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add(
        "",
        raw_device_count,
        radius=["40%", "55%"],
        center=[400,300],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="\nShare of devices across our sample"))
    .render('raw_device.html')
)



In [25]:
# plot of device_ban
device_ban_count = users.groupby('device')['ban'].sum().drop(labels = 'else')
device_ban_count = device_ban_count.reset_index().values.tolist()
device_ban_count

[['Apple', 1991.0],
 ['Huawei', 208.0],
 ['Nokia', 1.0],
 ['Oppo', 18.0],
 ['Samsung', 2.0],
 ['Vivo', 26.0],
 ['Xiaomi', 38.0],
 ['web', 724.0]]

In [242]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add(
        "",
        device_ban_count,
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="\nShare of the absolute amount of bans across devices"))
    .render("device_ban.html")
)

In [27]:
# raw_gender
raw_gender_count = users['gender'].value_counts()
raw_gender_count = raw_gender_count.reset_index().values.tolist()

# plot the gender graph
gender_ban = users.groupby('gender')['ban_binary'].sum()
gender_ban = gender_ban.reset_index().values.tolist()
gender_ban

[['f', 763], ['m', 791]]

In [28]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType
# female
list2 = [
    {"value": raw_gender_count[0][1], "percent":raw_gender_count[0][1] / (raw_gender_count[0][1] + raw_gender_count[1][1])},
    {"value": gender_ban[0][1], "percent": gender_ban[0][1] / (gender_ban[0][1] + gender_ban[1][1])}
]
#male
list3 = [
    {"value":raw_gender_count[1][1] , "percent": raw_gender_count[1][1] / (raw_gender_count[0][1] + raw_gender_count[1][1])},
    {"value": gender_ban[1][1], "percent": gender_ban[1][1] / (gender_ban[0][1] + gender_ban[1][1])}
]

c = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(['Raw','Censorship'])
    .add_yaxis("Female", list2, stack="stack1", category_gap="50%")
    .add_yaxis("Male", list3, stack="stack1", category_gap="50%")
    .set_series_opts(
        label_opts=opts.LabelOpts(
            position="right",
            formatter=JsCode(
                "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
            ),
        )
    )
    .render("gender.html")
)


In [29]:
# one-hot encode the data

from textblob import TextBlob
unique_locations = users['location'].unique()
unique_locations[16] = '未知'
recode_unique_locations = []

In [30]:
for i in unique_locations:
    blob = TextBlob('%s'%i)
    trans = str(blob.translate(to='en'))
    recode_unique_locations.append(trans)

In [31]:
recode_unique_locations

['overseas',
 'Beijing',
 'Hubei',
 'Zhejiang',
 'Guangdong',
 'Shanghai',
 'Sichuan',
 'Shaanxi',
 'Yunnan',
 'other',
 'Jiangsu',
 'Taiwan',
 'Shandong',
 'Tianjin',
 'Hunan',
 'Fujian',
 'unknown',
 'Guangxi',
 'Henan',
 'Chongqing',
 'Xinjiang',
 'Hong Kong',
 'Hebei',
 'Qinghai',
 'Liaoning',
 'Jilin',
 'Heilongjiang',
 'Anhui',
 'Guizhou',
 'Shanxi',
 'Gansu',
 'Jiangxi',
 'Inner Mongolia',
 'Ningxia',
 'Tibet',
 'Macao',
 'Hainan']

In [32]:
trans_dict = {}
for i in range(len(unique_locations)):
    trans_dict[unique_locations[i]] = recode_unique_locations[i]


train_user = users.loc[(users['ban_binary'] == True) | (users['ban_binary'] == False)]

del train_user['id']
del train_user['index']
train_user = train_user.reset_index(drop= True)
train_user.loc[train_user['location'] == 'unknown', ['location']] = '未知'
recode_location = []
for i in train_user['location']:
    recode_location.append(trans_dict[i])

train_user['location'] = recode_location

In [33]:
# one-hot encoding
train_user = pd.concat([train_user, pd.get_dummies(train_user['location'], prefix = 'location')], axis = 1)
train_user.drop(['location'], axis = 1, inplace = True)

In [34]:
train_user = pd.concat([train_user, pd.get_dummies(train_user['device'], prefix = 'device')], axis = 1)
train_user.drop(['device'], axis = 1, inplace = True)

In [35]:
train_user.loc[train_user['gender']=='m',['gender']] = 0
train_user.loc[train_user['gender']=='f',['gender']] = 1
train_user.loc[train_user['certified']==True,['certified']] = 1
train_user.loc[train_user['certified']==False,['certified']] = 0
train_user

,gender,fans,certified,ban,ban_binary,location_Anhui,location_Beijing,location_Chongqing,location_Fujian,location_Gansu,...,location_unknown,device_Apple,device_Huawei,device_Nokia,device_Oppo,device_Samsung,device_Vivo,device_Xiaomi,device_else,device_web
0,0,1,1,0.0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0.0,False,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,1,1,0.0,False,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,0.0,False,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,1,1,0.0,False,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5168,0,1,1,0.0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5169,1,1,1,6.0,True,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,1,1,1,0.0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5171,0,1,1,0.0,False,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
mean = train_user['ban'].mean()
std = train_user['ban'].std()

ban_class = []
ban_num = users['ban']
for i in ban_num:
    if i >= mean + std and i <mean + 2* std:
        ban_class.append('Probably')
    elif i>= mean + 2*std:
        ban_class.append('Very likely')
    elif i < mean + std and i >= mean:
        ban_class.append('Possibly')
    elif i < mean and i >= 0:
        ban_class.append('Rarely')
train_user['ban'] = ban_class

In [37]:
train_user_binary = train_user.drop(columns=['ban'])
train_user_binary.to_csv('train_binary.csv')

In [38]:
train_user_multi = train_user.drop(columns=['ban_binary'])
train_user_multi.to_csv('train_multi.csv')

In [39]:
#advanced analysis
users_advanced = users[(users['ban_binary'] == True) | (users['ban_binary'] == False)]
users_advanced = users_advanced.drop(columns=['id', 'index'])
users_advanced = users_advanced.reset_index(drop = True)
users_advanced

,gender,location,fans,certified,device,ban,ban_binary
0,m,海外,1,True,web,0.0,False
1,f,海外,0,False,Apple,0.0,False
2,m,北京,1,True,web,0.0,False
3,f,湖北,1,False,Apple,0.0,False
4,m,海外,1,True,Apple,0.0,False
...,...,...,...,...,...,...,...
5168,m,海外,1,True,web,0.0,False
5169,f,海外,1,True,web,6.0,True
5170,f,其他,1,True,web,0.0,False
5171,m,湖北,1,True,web,0.0,False


In [40]:
ban_class = []
ban_num = users_advanced['ban']
for i in ban_num:
    if i >= mean + std and i <mean + 2* std:
        ban_class.append('Probably')
    elif i>= mean + 2*std:
        ban_class.append('Very likely')
    elif i < mean + std and i >= mean:
        ban_class.append('Possibly')
    elif i < mean and i >= 0:
        ban_class.append('Rarely')
users_advanced['ban_extent'] = ban_class
users_advanced

,gender,location,fans,certified,device,ban,ban_binary,ban_extent
0,m,海外,1,True,web,0.0,False,Rarely
1,f,海外,0,False,Apple,0.0,False,Rarely
2,m,北京,1,True,web,0.0,False,Rarely
3,f,湖北,1,False,Apple,0.0,False,Rarely
4,m,海外,1,True,Apple,0.0,False,Rarely
...,...,...,...,...,...,...,...,...
5168,m,海外,1,True,web,0.0,False,Rarely
5169,f,海外,1,True,web,6.0,True,Very likely
5170,f,其他,1,True,web,0.0,False,Rarely
5171,m,湖北,1,True,web,0.0,False,Rarely


In [41]:
# certified
certified_ban = users_advanced.groupby('certified')['ban_binary'].value_counts()
# below is uncertified users
uncertified = certified_ban[0]
uncertified_false = uncertified[0]
uncertified_false = uncertified[1]
uncer = uncertified.values.tolist()
# below is certified users
certified = certified_ban[1]
certified_false = certified[0]
certified_true = certified[1]
cer = certified.values.tolist()
cer = [('False', cer[0]),('True', cer[1])]
uncer = [('False', uncer[0]),('True', uncer[1])]

In [229]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add("", cer)
    .set_colors(["blue", "green"])
    .set_global_opts(title_opts=opts.TitleOpts(title="Certified users ban case"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("certified_ban.html")
)

In [230]:
# as comparison
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add("", uncer)
    .set_colors(["blue", "green"])
    .set_global_opts(title_opts=opts.TitleOpts(title="Uncertified users ban case"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("uncertified_ban.html")
)

In [44]:
fans_ban = users_advanced.groupby('fans')['ban_binary'].value_counts()
unfan = fans_ban[0]
fan = fans_ban[1]
unfan = unfan.values.tolist()
fan = fan.values.tolist()
fan_false = [('False', fan[0]), ('True', fan[1])]
fan_true = [('False', unfan[0]), ('True', unfan[1])]

In [45]:
fans_ban

fans  ban_binary
0     False          544
      True           309
1     False         3075
      True          1245
Name: ban_binary, dtype: int64

In [48]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add("", fan_true)
    .set_colors(["orange", "red"])
    .set_global_opts(title_opts=opts.TitleOpts(title="\nUsers with less than 5000 followers ban case"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("unfan_ban.html")
)

In [47]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add("", fan_false)
    .set_colors(["orange", "red"])
    .set_global_opts(title_opts=opts.TitleOpts(title="Users with over 5000 followers ban case"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    .render("fan_ban.html")
)

In [49]:
location_extent = users_advanced.groupby('location')['ban_extent'].value_counts()
location_extent = location_extent.to_frame()
# get the top 5 areas where bans appear most
top_5_area = users_advanced.groupby('location')['ban'].count().sort_values(ascending=False).reset_index()
top_5_area = top_5_area[top_5_area['location'] != '其他']

top_5_area = top_5_area.head(5)
location_5 = top_5_area['location']

In [50]:
users_filtered = users_advanced[users_advanced['location'].isin(location_5)]
location_extent = users_filtered.groupby('location')['ban_extent'].value_counts().reset_index(name='counts')
location_extent

,location,ban_extent,counts
0,上海,Rarely,415
1,上海,Possibly,152
2,上海,Probably,19
3,上海,Very likely,16
4,北京,Rarely,916
5,北京,Possibly,294
6,北京,Very likely,50
7,北京,Probably,49
8,广东,Rarely,247
9,广东,Possibly,91


In [51]:
extent = location_extent['ban_extent'].unique()
count = []
for i in extent:
    count.append(location_extent.loc[location_extent['ban_extent'] == i]['counts'])

In [52]:
for i in range(len(count)):
    count[i] = count[i].to_numpy()

In [53]:
list1 = []
list2 = []
list3 = []
list4 = []
list5 = []
for i in range(len(count)):
    list1.append(count[i][0])
    list2.append(count[i][1])
    list3.append(count[i][2])
    list4.append(count[i][3])
    list5.append(count[i][4])

list1 = np.array(list1)
list1 = list1/list1.sum()
list1 = list1.tolist()
list2 = np.array(list2)
list2 = list2/list2.sum()
list2 = list2.tolist()
list3 = np.array(list3)
list3 = list3/list3.sum()
list3 = list3.tolist()
list4 = np.array(list4)
list4 = list4/list4.sum()
list4 = list4.tolist()
list5 = np.array(list5)
list5 = list5/list5.sum()
list5 = list5.tolist()

In [54]:
extent = extent.tolist()

In [55]:
name_location = location_extent['location'].unique()
name_location = name_location.tolist()

In [58]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
Faker.choose()

c = (
    Bar()
    .add_xaxis(extent)
    .add_yaxis("%s"%name_location[0], list1)
    .add_yaxis("%s"%name_location[1], list2)
    .add_yaxis("%s"%name_location[2], list3)
    .add_yaxis("%s"%name_location[3], list4)
    .add_yaxis("%s"%name_location[4], list5)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="\nTop 5 areas ban type distribution in percentage", subtitle=""),
        brush_opts=opts.BrushOpts(),
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .render("ban_topcities.html")
)


In [204]:
# heat map based on the person
location_ban_binary = users_advanced.groupby('location')['ban_binary'].count()
raw_users = users_advanced.groupby('location')['ban_binary'].value_counts().reset_index(name = 'count')
raw_users = raw_users.loc[raw_users['ban_binary']==True]
count = raw_users['count']
raw_users = raw_users[~raw_users['location'].isin(['unknown', '其他','海外'])]
count = raw_users['count']
count = count.reset_index(drop = True)

location_ban_binary = location_ban_binary.drop(labels=['unknown', '其他','海外'])
location_ban_binary = location_ban_binary.to_frame().reset_index()

raw_users = raw_users.reset_index(drop=True)
count_ = []
for i in range(len(raw_users['count'])):
    location = raw_users['location'][i]
    count_.append(int(location_ban_binary[location_ban_binary['location'] == location]['ban_binary']))

raw_users['total'] = count_
raw_users = raw_users[raw_users['total']>=49]
raw_user_location = raw_users['location'].to_frame().reset_index(drop = True)
raw_user_location = raw_user_location.to_numpy().flatten()

raw_users['percentile'] = raw_users['count'] / raw_users['total']*100


raw_users = raw_users.drop(columns=['ban_binary','count','total'])
raw_users = raw_users.reset_index(drop=True).values.tolist()
raw_users

[['上海', 31.06312292358804],
 ['北京', 30.02291825821238],
 ['四川', 31.967213114754102],
 ['广东', 32.880434782608695],
 ['江苏', 34.53237410071942],
 ['浙江', 25.757575757575758],
 ['湖北', 33.33333333333333],
 ['湖南', 22.807017543859647],
 ['福建', 20.73170731707317],
 ['重庆', 22.448979591836736],
 ['陕西', 38.775510204081634],
 ['香港', 43.13725490196079]]

In [166]:
#

[['上海', 0.3106312292358804],
 ['云南', 0.125],
 ['内蒙', 0.36363636363636365],
 ['北京', 0.3002291825821238],
 ['台湾', 0.19047619047619047],
 ['吉林', 0.42105263157894735],
 ['四川', 0.319672131147541],
 ['天津', 0.2857142857142857],
 ['安徽', 0.3333333333333333],
 ['山东', 0.391304347826087],
 ['山西', 0.5],
 ['广东', 0.328804347826087],
 ['广西', 0.2],
 ['新疆', 0.5],
 ['江苏', 0.34532374100719426],
 ['江西', 0.2727272727272727],
 ['河北', 0.47619047619047616],
 ['河南', 0.15625],
 ['浙江', 0.25757575757575757],
 ['湖北', 0.3333333333333333],
 ['湖南', 0.22807017543859648],
 ['澳门', 0.6666666666666666],
 ['甘肃', 0.375],
 ['福建', 0.2073170731707317],
 ['西藏', 0.5],
 ['贵州', 0.125],
 ['辽宁', 0.42857142857142855],
 ['重庆', 0.22448979591836735],
 ['陕西', 0.3877551020408163],
 ['香港', 0.43137254901960786],
 ['黑龙江', 0.07692307692307693]]

In [185]:
from pyecharts import options as opts
from pyecharts.charts import Map


c = (
    Map()
    .add("Percentile", raw_users, "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="\nPercentile of users been banned once or more"),
        visualmap_opts=opts.VisualMapOpts(max_=50),
    )
    .render('percentile_ban_location.html')
)

In [211]:
#
data = []
for i in range(len(raw_users)):
    data.append(raw_users[i][1])

[31.06312292358804,
 30.02291825821238,
 31.967213114754102,
 32.880434782608695,
 34.53237410071942,
 25.757575757575758,
 33.33333333333333,
 22.807017543859647,
 20.73170731707317,
 22.448979591836736,
 38.775510204081634,
 43.13725490196079]

In [222]:
#
raw_user_location

['上海', '北京', '四川', '广东', '江苏', '浙江', '湖北', '湖南', '福建', '重庆', '陕西', '香港']

In [228]:
import pyecharts.options as opts
from pyecharts.charts import Bar

bar = (
    Bar()
    .add_xaxis(raw_user_location)
    .add_yaxis('Loation', data)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title='\nPercentile of users whose reposts had been censored',
                                               title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                     xaxis_opts=opts.AxisOpts(name='Location'),
                     yaxis_opts=opts.AxisOpts(name='Unit(percentile)'))
)
bar.render('bar_location_ban.html')

'C:\\Users\\pasaa\\Desktop\\440_data_processing\\bar_location_ban.html'

In [187]:
#
raw_users

[['上海', 31.06312292358804],
 ['北京', 30.02291825821238],
 ['四川', 31.967213114754102],
 ['广东', 32.880434782608695],
 ['江苏', 34.53237410071942],
 ['浙江', 25.757575757575758],
 ['湖北', 33.33333333333333],
 ['湖南', 22.807017543859647],
 ['福建', 20.73170731707317],
 ['重庆', 22.448979591836736],
 ['陕西', 38.775510204081634],
 ['香港', 43.13725490196079]]

In [250]:
per_device = users.groupby('device')['ban_binary'].value_counts().drop(labels = 'else').reset_index(name = "count")
per_device = per_device[per_device['ban_binary'] == True].reset_index(drop=True)

per_device = per_device.drop(columns='ban_binary').values.tolist()
per_device



[['Apple', 914],
 ['Huawei', 104],
 ['Nokia', 1],
 ['Oppo', 10],
 ['Samsung', 2],
 ['Vivo', 7],
 ['Xiaomi', 23],
 ['web', 330]]

In [252]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker

c = (
    Pie()
    .add(
        "",
        per_device,
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="\nShare of devices users used to repost banned contents"))
    .render("device_banper.html")
)

In [243]:
device_ban_count

[['Apple', 1991.0],
 ['Huawei', 208.0],
 ['Nokia', 1.0],
 ['Oppo', 18.0],
 ['Samsung', 2.0],
 ['Vivo', 26.0],
 ['Xiaomi', 38.0],
 ['web', 724.0]]